## 最適化問題

$$ \min*h \|\bm{g}-F^\top \bm{h}\|\_2^2+\lambda_1\|\bm{h}\|*{1,2}^2 + \lambda*2\|D\bm{h}\|*{1,2}$$


In [2]:
import os
import re
import random
import math
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import norm, spsolve
from typing import Callable, Union
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from PIL import Image

In [2]:
# パラメータ設定
n = 64
m = 128
LAMBDA1 = 1
LAMBDA2 = 1
SEED = 2
RATIO = 0.1
DATA_PATH = "../../OneDrive - m.titech.ac.jp/Lab/data"
IMG_NAME = "hadamard"
DIRECTORY = DATA_PATH + "/240718"
SETTING = f"{IMG_NAME}_l1_p-{int(100*RATIO)}_lmd1-{int(LAMBDA1)}_lmd2-{int(LAMBDA2)}"

In [ ]:
def matrix2vector(matrix):
    return matrix.T.flatten()


def vector2matrix(vector, s, t):
    return vector.reshape(s, t).T


def mult_mass(F, h, M):
    N = F.shape[1]
    h = h.reshape(N, M)
    res = F @ h
    return res.flatten()


def images_to_matrix(folder_path, convert_gray=True, rand=True, ratio=RATIO):
    files = os.listdir(folder_path)
    files.sort(key=lambda f: int(re.search(f"{IMG_NAME}_(\d+).png", f).group(1)))
    if rand:
        random.seed(SEED)
        random.shuffle(files)

    total_files = len(files)
    number_of_files_to_load = int(total_files * ratio)
    selected_files = files[:number_of_files_to_load]
    selected_files.sort(key=lambda f: int(re.search(f"{IMG_NAME}_(\d+).png", f).group(1)))

    images = []
    use_list = []

    for file in selected_files:
        index = int(re.sub(r"\D", "", file))
        use_list.append(index)
        img = Image.open(os.path.join(folder_path, file))
        if convert_gray:
            img = img.convert("L")
        img_array = np.asarray(img).flatten()
        img_array = img_array / 255
        images.append(img_array)

    return np.column_stack(images), use_list

In [ ]:
def prox_l1(x: np.ndarray, alpha: float) -> np.ndarray:
    """Proximal operator for L1 norm."""
    return np.sign(x) * np.maximum(np.abs(x) - alpha, 0)


def prox_l122(y: np.ndarray, gamma: float) -> np.ndarray:
    Y = vector2matrix(y, n**2, m**2)
    N = Y.shape[1]
    l1_norms = np.sum(np.abs(Y), axis=1)
    factor = (2 * gamma) / (1 + 2 * gamma * N)
    X = np.zeros_like(Y)
    X = np.sign(Y) * np.maximum(np.abs(Y) - factor * l1_norms[:, np.newaxis], 0)
    return matrix2vector(X)


def prox_conj(prox: Callable[[np.ndarray, float], np.ndarray], x: np.ndarray, alpha: float) -> np.ndarray:
    """Conjugate proximal operator."""
    return x - alpha * prox(x / alpha, 1 / alpha)


def grad_h(h: np.ndarray, X: np.ndarray, g: np.ndarray) -> np.ndarray:
    """Gradient of ||g - Xh||_2^2."""
    return 2 * X.T @ (X @ h - g)


def primal_dual_splitting(
    X: np.ndarray,
    D: np.ndarray,
    g: np.ndarray,
    lambda1: float,
    lambda2: float,
    max_iter: int = 1000,
    tol: float = 1e-2,
) -> tuple[np.ndarray, dict]:
    """
    Solve the optimization problem:
    min_h ||g-Fh||_2^2 + lambda1 P(h) + lambda2 ||Dh||_{1,2}
    using the primal-dual splitting method.

    Args:
        X (np.ndarray): Matrix X in the problem formulation.
        D (np.ndarray): Gradient matrix D.
        g (np.ndarray): Vector g in the problem formulation.
        lambda1 (float): Regularization parameter for L1 norm of h.
        lambda2 (float): Regularization parameter for L1 norm of Dh.
        h_init (np.ndarray, optional): Initial guess for h. If None, initialized with zeros.
        y_init (np.ndarray, optional): Initial guess for y. If None, initialized with zeros.
        max_iter (int): Maximum number of iterations.
        tol (float): Tolerance for convergence.

    Returns:
        tuple[np.ndarray, dict]: Solution h and a dictionary containing additional information.
    """
    MK, MN = X.shape
    h = np.zeros(MN)
    y = np.zeros(D.shape[0])

    # Compute Lipschitz constant of grad_f
    L = norm(X, 2) ** 2

    tau = 1.0 / L
    sigma = 1.0 / (norm(D, 2) ** 2)

    objective_values = []
    for k in range(max_iter):
        h_old = h.copy()
        y_old = y.copy()

        # Update primal variable h
        grad = grad_h(h_old, X, g)
        h = prox_l1(h - tau * (grad - D.T @ y), tau * lambda1)

        # Update dual variable y
        y = prox_conj(prox_l1, y + sigma * D @ (2 * h - h_old), sigma / lambda2)

        # Compute objective value
        obj_value = np.linalg.norm(g - X @ h) ** 2 + lambda1 * np.linalg.norm(h, 1) + lambda2 * np.linalg.norm(D @ h, 1)
        objective_values.append(obj_value)

        # Check convergence
        primal_residual = np.linalg.norm(h - h_old) / max(np.linalg.norm(h), 1e-8)
        dual_residual = np.linalg.norm(y - y_old) / max(np.linalg.norm(y), 1e-8)
        print(f"iter={k}, obj={obj_value:.4f}, primal_res={primal_residual:.4f}, dual_res={dual_residual:.4f}")
        if primal_residual < tol and dual_residual < tol:
            break

    info = {
        "iterations": k + 1,
        "objective_values": objective_values,
        "final_objective": objective_values[-1],
        "primal_residual": primal_residual,
        "dual_residual": dual_residual,
    }

    return h, info

In [ ]:
# load images
G, use = images_to_matrix(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/")
F, _ = images_to_matrix(f"{DATA_PATH}/{IMG_NAME}{n}_input/")
print("K=", F.shape[1])
white_img = Image.open(f"{DATA_PATH}/{IMG_NAME}{n}_cap_R_230516_128/{IMG_NAME}_1.png")
white_img = white_img.convert("L")
white_img = np.asarray(white_img) / 255
white = white_img.flatten()
white = white[:, np.newaxis]
H1 = np.tile(white, F.shape[1])
F_hat = 2 * F - 1
G_hat = 2 * G - H1

g = matrix2vector(G_hat)

In [20]:
m = 4
M = m**2

Di = sparse.lil_matrix((M, M))
Dj = sparse.lil_matrix((M, M))
Dk = sparse.lil_matrix((M, 2 * M))
Dl = sparse.lil_matrix((M, 3 * M))

for i in range(m):
    for j in range(m):
        index = i * m + j
        if i < n - 1:
            # 内部ピクセルの場合
            Di[index, index] = 1
            Di[index, index + m] = -1

for i in range(m):
    for j in range(m):
        index = i * m + j
        if j < m - 1:
            # 内部ピクセルの場合
            Dj[index, index] = 1
            Dj[index, index + 1] = -1


print(Di.toarray())
print(Dj.toarray())

[[ 1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [ ]:
D = sparse.eye(n, n, format="csc")

In [ ]:
h, info = primal_dual_splitting(F_hat, D, g, LAMBDA1, LAMBDA2, max_iter=500)